In [ ]:
import pandas as pd 
import numpy as np


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving chem data2hj.xlsx to chem data2hj.xlsx


In [ ]:
data=pd.read_excel('chem data2hj.xlsx')

In [ ]:
data

,Conc. Of Tartrazine(mg/L),Ads Dose(g/L),Time(min),Absorbance,Concentration (mg/L),Removal (mg/L),% Removal
0,10,1,0,0.4135,4.135,5.865,58.65
1,10,1,15,0.0229,0.229,9.771,97.71
2,10,1,30,0.0153,0.153,9.847,98.47
3,10,1,45,0.0152,0.152,9.848,98.48
4,10,1,60,0.0140,0.140,9.860,98.60
5,10,1,120,0.0111,0.111,9.889,98.89
6,20,1,0,0.7855,7.855,2.145,21.45
7,20,1,15,0.0248,0.248,9.752,97.52
8,20,1,30,0.0179,0.179,9.821,98.21
9,20,1,45,0.0146,0.146,9.854,98.54


In [ ]:
x=data.iloc[:,0:3]
y=data.iloc[:,-1]

In [ ]:
y

0      58.65
1      97.71
2      98.47
3      98.48
4      98.60
5      98.89
6      21.45
7      97.52
8      98.21
9      98.54
10     98.73
11     98.89
12      1.58
13     74.45
14     91.20
15     95.01
16     99.76
17    100.00
18      0.76
19      1.82
20     12.57
21     34.25
22     39.35
23     55.37
24      0.08
25     -6.87
26     16.43
27     33.52
28     34.76
29     44.10
Name:  % Removal , dtype: float64

In [ ]:
x

,Conc. Of Tartrazine(mg/L),Ads Dose(g/L),Time(min)
0,10,1,0
1,10,1,15
2,10,1,30
3,10,1,45
4,10,1,60
5,10,1,120
6,20,1,0
7,20,1,15
8,20,1,30
9,20,1,45


In [ ]:
import tensorflow as tf

In [ ]:
pip install keras-tuner

     |████████████████████████████████| 98 kB 3.5 MB/s 


In [ ]:
import kerastuner
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.model_selection import train_test_split as tts

In [ ]:
Xtrain,Xtest,ytrain,ytest=tts(x,y,test_size=0.1,random_state=42)

In [ ]:
def model(hp):
  build_model=Sequential()
  build_model.add(Dense(units=hp.Int('n',min_value=16,max_value=512,step=16),activation='relu',input_dim=Xtrain.shape[1]))
  for i in range(hp.Int('nlayers',min_value=2,max_value=10)):
    build_model.add(Dense(units=hp.Int('units',min_value=16,max_value=256,step=16),activation='relu'))
  build_model.add(Dense(units=1,activation='linear'))
  build_model.compile(optimizer=Adam(hp.Choice('lr',[1e-2,1e-3,1e-4])),loss='mean_absolute_error',metrics=['mean_absolute_error'])
  return build_model

In [ ]:
tuner=RandomSearch(model,objective='val_mean_absolute_error',max_trials=15,executions_per_trial=3,directory='output',project_name='chemann')

In [ ]:
tuner.search(Xtrain,ytrain,epochs=50,validation_data=(Xtest,ytest),batch_size=2)

Trial 15 Complete [00h 00m 30s]
val_mean_absolute_error: 10.920831521352133

Best val_mean_absolute_error So Far: 5.171207427978516
Total elapsed time: 00h 07m 03s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_model=tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 304)               1216      
                                                                 
 dense_1 (Dense)             (None, 80)                24400     
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 80)                6480      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 80)                6480      
                                                                 
 dense_6 (Dense)             (None, 80)                6

In [ ]:
ypred=best_model.predict(Xtest)

In [ ]:
ypred

array([[35.27872 ],
       [95.8357  ],
       [57.427452]], dtype=float32)

In [ ]:
ytest

27    33.52
15    95.01
23    55.37
Name:  % Removal , dtype: float64

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
print(mae(ytest,ypred))

1.5472913106282533
